# IntegrAI Demo
In this demonstration, we will use a subset of the ImageNet and a pre-trained vision model, and try to discover and describe where it does really well and where it does badly

In [3]:
import sys
sys.path.append('./src')
from teacher_methods.teacher_gen import *
from utils.utils import loss_01
from utils.metrics_hai import compute_metrics
import numpy as np
import meerkat as mk
import os
from collections import Counter
import ipyplot
from meerkat.datasets.imagenette import download_imagenette
import torch
import json
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
from utils.utils import loss_01
from describers.itterative_describe import *
from torchvision.models import resnet18
import torchvision.transforms as transforms


We will download a subset of the imagenet dataset and use a pre-trained model.

This follows the steps in http://meerkat.wiki/docs/start/tutorials/tutorial-data-frames.html with minimal modifications

# Data and AI Model prediction

Feel free to quickly run but not read this section in any detail

Downloads the dataset to folder ./downloads

In [4]:
dataset_dir = "./downloads"
os.makedirs(dataset_dir, exist_ok=True)
download_imagenette(dataset_dir, overwrite=False);

from imagenet_labels_inf import *
imagenet_labels = imagenet_labels

This is a meerkat dataframe which will be useful here

In [5]:
# Create a `DataFrame`
df = mk.from_csv("./downloads/imagenette2-160/imagenette.csv")

# Create an `ImageColumn`` and add it to the `DataFrame`
df["img"] = mk.image(
    df["img_path"], 
    base_dir=os.path.join(dataset_dir, "imagenette2-160")
)
df = df[df['split']=='valid']

Feel free to exchange this with any other model

In [6]:
model = resnet18(weights="ResNet18_Weights.IMAGENET1K_V1")

This code below just gets the prediction of the model on the dataset

In [7]:
# Define transform
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]),
])

# Create new column with transform 
df["input"] = df["img"].defer(transform)
# Define forward hook in ActivationExtractor class
class ActivationExtractor:
    """Extracting activations a targetted intermediate layer"""

    def __init__(self):
        self.activation = None

    def forward_hook(self, module, input, output):
        self.activation = output

# Register forward hook
extractor = ActivationExtractor()
model.layer4.register_forward_hook(extractor.forward_hook);

# 1. Move the model to GPU, if available
# device = 0
device = "cuda:0"
model.to(device).eval()
# 2. Define a function that runs a forward pass over a batch 
@torch.no_grad()
def predict(input: mk.TensorColumn):

    x: torch.Tensor = input.data.to(device)  # We get the underlying torch tensor with `data` and move to GPU 
    out: torch.Tensor = model(x)  # Run forward pass

    # Return a dictionary with one key for each of the new columns. Each value in the
    # dictionary should have the same length as the batch. 
    return {
        "pred": out.cpu().numpy().argmax(axis=-1),
        "probs": torch.softmax(out, axis=-1).cpu(),
    }
# 3. Apply the update. Note that the `predict` function operates on batches, so we set 
# `batched=True`. Also, the `predict` function only accesses the "input" column, by 
# specifying that here we instruct update to only load that one column and skip others 
pred_df = df.map(function=predict, is_batched_fn=True, batch_size=32)
df = mk.concat([df, pred_df], axis="columns")

we will convert the predictions and labels to a scale from 0 to 9 for ease of use and keep this handy:

{'cassette player': 0,
 'garbage truck': 1,
 'tench': 2,
 'english springer spaniel': 3,
 'church': 4,
 'parachute': 5,
 'french horn': 6,
 'chainsaw': 7,
 'golf ball': 8,
 'gas pump': 9}

In [8]:

# Given array of labels
labels = Counter(df['label'])
labels_simple = df['label']
preds_simple = [imagenet_labels[df["pred"][i]] for i in range(len(df["pred"]))]
# Create a mapping from strings to integers (0 to number of unique labels - 1)
label_to_int = {label: i for i, label in enumerate(labels.keys())}

# Convert the labels to integers
labels_simple = [label_to_int[label] for label in labels_simple]

for i in range(len(preds_simple)):
    if preds_simple[i] in label_to_int:
        preds_simple[i] = label_to_int[preds_simple[i]]
    else:
        preds_simple[i] = np.random.randint(0, len(label_to_int))
preds = np.array(preds_simple)
labels = np.array(labels_simple)

In [9]:
print(f' accuracy of AI is {np.mean(preds == labels)}')

 accuracy of AI is 0.5880254777070064


# IntegrAI-Discover

The first step in the process is to embed our images in a cross-modal embedding space.

We use CLIP for this reason.

Note: there exists better versions of CLIP models than the ones we use here!

In [10]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# need to put it on gpu if needed

image_embeddings = []
text_embeddings = []
BATCH_SIZE = 15
number_of_batches = math.ceil(len(df['img_path']) / BATCH_SIZE)

for batch in tqdm(range(number_of_batches)):
    images = []
    texts = []
    for i in range(BATCH_SIZE):
        if batch * BATCH_SIZE + i < len(df['img_path']):
            images.append(Image.open("./downloads/imagenette2-160/" + df['img_path'][batch * BATCH_SIZE + i]))
            texts.append("") # this is just to play nice with the model
    inputs = processor(
        text=texts, images=images, return_tensors="pt", padding=True
    )
    with torch.no_grad():
        outputs = model(**inputs)
    for i in range(len(texts)):
        image_embeddings.append(outputs.image_embeds[i])
# convert to numpy
image_embeddings = torch.stack(image_embeddings).numpy()


100%|███████████████████████████████████████████| 262/262 [02:41<00:00,  1.62it/s]


Before we discover regions, we need to define what our prior is.

The prior is essentially what are our prior beliefs about the AI that we want to disprove.

In this demo, you can play around with two kinds of priors easily (each with a parameter):

- see regions where AI has accuracy higher than X%: set SEE_GOOD_REGIONS = True and set HOW_GOOD = X.

  **This is essentially to tell us where to trust the AI model**

- see regions where AI has accuracy lower than X%: set SEE_GOOD_REGIONS = False and set HOW_GOOD = X.

  **This is essentially to tell us where *NOT* to trust the AI model**


In [17]:
SEE_GOOD_REGIONS = True 
HOW_GOOD = 0.7
hum_preds = []
np.random.seed(0)

# hum_preds denote the human's label predictions if availabl
# here we make it to be synthetic where the human has HOW_GOOD accuracy uniformly across all points
for true_label in labels:
    if np.random.rand() <= HOW_GOOD:
        hum_preds.append(true_label)
    else:
        possible_labels = [label for label in range(len(label_to_int)) if label != true_label]
        hum_preds.append(np.random.choice(possible_labels))
hum_preds = np.array(hum_preds)

if SEE_GOOD_REGIONS:
    # all 0s: never trust the AI
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[1,0]) for i in range(len(df['img']))])
else:
    # all 1s: always trust the AI
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[0,1]) for i in range(len(df['img']))])


alpha = 0.5 # CONSISTENCY OF REGION TAKEAWAY
beta_high = 0.05 # MAX REGION SIZE
beta_low = 0.0 # MIN REGION SIZE
lr = 0.001
delta = 1
metric_y = loss_01


In [18]:
# HOW MANY REGIONS TO SEE?
max_regions = 1
# this is IntegrAI-Describe
teacher_gen = TeacherGenerative(image_embeddings, labels,
                                hum_preds, preds
                                , prior_rejector_preds, metric_y, max_regions, alpha, beta_high, beta_low, delta)
teacher_gen.lr = lr
teacher_gen.fit()

  0%|                                                       | 0/2 [00:16<?, ?it/s]


The get_region_labels helper annotates the data with which region it belongs to. Region 0 is the background so region 1 is what we want

In [19]:
# which points are in the region? 0 is background region always
region_labels = teacher_gen.get_region_labels(image_embeddings, prior_rejector_preds)
defer_preds = teacher_gen.get_defer_preds(image_embeddings, prior_rejector_preds)
print(f' accuracy of AI in region found is {np.mean(preds[region_labels==1] == labels[region_labels==1])}')
print(f' size of regions is {Counter(region_labels)}')

 accuracy of AI in region found is 0.9847715736040609
 size of regions is Counter({0.0: 3728, 1.0: 197})


Let's look at images in the region!

In [20]:
image_names = df['img_path'].to_numpy()[region_labels==1]
labels_imgs = df['label'].to_numpy()[region_labels==1]
for i in range(len(image_names)):
    image_names[i] = "./downloads/imagenette2-160/" + image_names[i]
#ipyplot.plot_images(image_names, max_images=50, img_width=150)
ipyplot.plot_class_tabs(image_names, labels_imgs, max_imgs_per_tab=200, img_width=100,  show_url=False)

We find that all the points in the region belong to the parachute class!

Let's see if there is something special about these pictures from the parachute class overall.

In [21]:
import ipyplot
image_names = df['img_path'].to_numpy()[df['label'] == 'parachute']
labels_imgs = df['label'].to_numpy()[df['label'] == 'parachute']
for i in range(len(image_names)):
    image_names[i] = "./downloads/imagenette2-160/" + image_names[i]
#ipyplot.plot_images(image_names, max_images=50, img_width=150)
ipyplot.plot_class_tabs(image_names, labels_imgs, max_imgs_per_tab=400, img_width=100,  show_url=False)

We can see that the parachutes in our region always have a clear blue sky in the background, while those in the overall class have more variety with parachutes on the ground and so.

Note that the AI is not perfect on the parachute class. AI has 81% accuracy on the class, which means it has approximately 60% accuracy on the class parachute excluding the region found.

In [23]:
print(f' accuracy of AI in label parachute {np.mean(preds[labels ==5] == labels[labels==5])}')

 accuracy of AI in label parachute 0.8128205128205128


# IntegrAI-Describe

Now that we found the region, let's see if we can describe it in some way.

The first thing we need to do is describe each image in our dataset with a captioning algorithm.

In the paper, we used the MS-COCO dataset where captions are already provided for example or the BDD dataset where we can derive a caption from metadata.

Here we use a SOTA captioning algorithm instead:

In [26]:
from tqdm import tqdm
captions = []
for i in tqdm(range(len(df['img_path']))):
    output = image_to_text("./downloads/imagenette2-160/" + df['img_path'][i])
    captions.append(output[0]['generated_text'])
captions = np.array(captions)

  0%|                                                    | 0/3925 [00:00<?, ?it/s]/opt/conda/mozannar/envs/teach_vision/lib/python3.8/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████| 3925/3925 [22:21<00:00,  2.93it/s]


This code gets the scores of each point belonging to the region:

In [42]:
region_scores = teacher_gen.get_region_labels_probs(image_embeddings)
region_scores_new = []
for i in range(len(region_scores)):
    temp = [0]
    for j in range(len(region_scores[i])):
        temp.append(region_scores[i][j])
    region_scores_new.append(np.array(temp))
region_scores = np.array(region_scores_new)

In [24]:
# YOUR OPENAI KEY
keyfile = open("./keys.txt", "r")
# read the file
key = keyfile.read()


In [ ]:
itt_desriber = IterativeRegionDescribe(captions, image_embeddings, region_scores, region_labels, key, get_text_embeddings,
                                       n_rounds = 2,  initial_positive_set_size = 100, initial_negative_set_size = 20)
descriptions = itt_desriber.describe_region(1)


In [53]:
print(descriptions[0][-1])

The region consists of descriptions of people flying kites in the air, specifically with parachutes or multiple people, contrasted with other flying objects like airplanes.


Our algorithm finds the following description:

`The region consists of descriptions of people flying kites in the air, specifically with parachutes or multiple people, contrasted with other flying objects like airplanes.
`


# Finding more than one region

In [54]:
SEE_GOOD_REGIONS = False 
HOW_GOOD = 0.4
hum_preds = []
np.random.seed(0)

# hum_preds denote the human's label predictions if availabl
# here we make it to be synthetic where the human has HOW_GOOD accuracy uniformly across all points
for true_label in labels:
    if np.random.rand() <= HOW_GOOD:
        hum_preds.append(true_label)
    else:
        possible_labels = [label for label in range(len(label_to_int)) if label != true_label]
        hum_preds.append(np.random.choice(possible_labels))
hum_preds = np.array(hum_preds)

if SEE_GOOD_REGIONS:
    # all 0s: never trust the AI
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[1,0]) for i in range(len(df['img']))])
else:
    # all 1s: always trust the AI
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[0,1]) for i in range(len(df['img']))])


alpha = 0.5 # CONSISTENCY OF REGION TAKEAWAY
beta_high = 0.05 # MAX REGION SIZE
beta_low = 0.0 # MIN REGION SIZE
lr = 0.001
delta = 1
metric_y = loss_01


In [55]:
# HOW MANY REGIONS TO SEE?
max_regions = 5
# this is IntegrAI-Describe
teacher_gen = TeacherGenerative(image_embeddings, labels,
                                hum_preds, preds
                                , prior_rejector_preds, metric_y, max_regions, alpha, beta_high, beta_low, delta)
teacher_gen.lr = lr
teacher_gen.fit()

 40%|██████████████████▍                           | 4/10 [01:10<01:46, 17.72s/it]


In [69]:
# which points are in the region? 0 is background region always
region_labels = teacher_gen.get_region_labels(image_embeddings, prior_rejector_preds)
defer_preds = teacher_gen.get_defer_preds(image_embeddings, prior_rejector_preds)
print(f' size of regions is {Counter(region_labels)}')
for i in range(max_regions + 1):
    if i == 0:
        print(f'accuracy of AI in background region {np.mean(preds[region_labels==i] == labels[region_labels==i])}')
    else:
        print(f'accuracy of AI in region {i} found is {np.mean(preds[region_labels==i] == labels[region_labels==i])}')


 size of regions is Counter({0.0: 3143, 2.0: 197, 4.0: 180, 1.0: 151, 3.0: 149, 5.0: 105})
accuracy of AI in background region 0.7120585427935094
accuracy of AI in region 1 found is 0.059602649006622516
accuracy of AI in region 2 found is 0.06091370558375635
accuracy of AI in region 3 found is 0.04697986577181208
accuracy of AI in region 4 found is 0.12777777777777777
accuracy of AI in region 5 found is 0.18095238095238095


In [ ]:
image_names = df['img_path'].to_numpy()
for i in range(len(image_names)):
    image_names[i] = "./downloads/imagenette2-160/" + image_names[i]
#ipyplot.plot_images(image_names, max_images=50, img_width=150)
ipyplot.plot_class_tabs(image_names, region_labels, max_imgs_per_tab=20, img_width=100,  show_url=False)